## 참고자료

https://www.sheshbabu.com/posts/rust-error-handling/

## 1. 러스트 에러 

- 패닉은 러스트에서의 에러를 의미 

In [2]:
fn main() {
    panic!("crash and burn");
}

In [3]:
main()

thread '<unnamed>' panicked at 'crash and burn', src/lib.rs:3:5
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: ctx::main
   3: _run_user_code_2
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


## 러스트 인덱스 에러 

- 인덱스 검색할 때 인덱스 범위를 벗어날 경우 에러가 발생

In [4]:
fn main() {
    let v = vec![1, 2, 3];

    v[99];
}

In [5]:
main()

thread '<unnamed>' panicked at 'index out of bounds: the len is 3 but the index is 99', src/lib.rs:5:5
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::panicking::panic_bounds_check
   3: <unknown>
   4: <unknown>
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


## 2. Result를 사용해 에러 처리하기 

In [ ]:
Result는 Rust 표준 라이브러리에 정의된 열거형(Enumeration) 타입으로, 함수나 메서드 호출 결과가 성공인지 실패인지를 나타내는 타입입니다. 
Result는 두 개의 제네릭 매개변수를 가지며, Ok(T) 또는 Err(E) 중 하나의 값을 가집니다. 
여기서 T는 성공 결과를 나타내는 값의 타입이며, E는 실패 결과를 나타내는 값의 타입입니다.

## Recoverable Errors with Result

- 별도의 예외처리 방식이 없다. 
- 대신 이넘 Result 로 처리하고 정상이면 OK이고 아니면 Err를 발생시킴.
- OK 내의 값은 정상처리한 결과이고 Err의 값은 에러를 발생하는 메시지를 의미한다.


### Rust에서는 Result라는 것을 반환으로 예외처리하기 

 - Result<T, E> 유형은 성공 값은 Ok(T), 오류 값은 Err(E)의 두 가지 변형을 가진 열거형입니다:

## result 이넘 알아보기

- Result<T, E> 유형은 성공 값인 Ok(T) 또는 오류 값인 Err(E)의 두 가지 변형이 있는 열거형입니다:


### enum Result<T, E> {
###   Ok(T),
###   Err(E),
### }

In [6]:
use std::fs::File;

println!(" {:?} ",File::open("hello.txt"));

 Err(Os { code: 2, kind: NotFound, message: "No such file or directory" }) 


In [7]:
use std::fs::File;

fn main() {
    let greeting_file_result = File::open("hello1.txt");
    
    println!(" 파일처리 {:?}", greeting_file_result);
}

In [8]:
main()

 파일처리 Err(Os { code: 2, kind: NotFound, message: "No such file or directory" })


()

### 파일을 읽으면 result 로 반환한다.

- 이를 match를 사용해서 정상과 오류처리를 한다.

In [9]:
use std::fs::File;

fn main() {
    let greeting_file_result = File::open("hello2.txt");

    let greeting_file = match greeting_file_result {
        Ok(file) => file,
        Err(error) => panic!("Problem opening the file: {:?}", error),
    };
}

In [10]:
main()

thread '<unnamed>' panicked at 'Problem opening the file: Os { code: 2, kind: NotFound, message: "No such file or directory" }', src/lib.rs:8:23
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: <unknown>
   3: <unknown>
   4: <unknown>
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


## Matching on Different Errors

In [11]:
use std::fs::File;
use std::io::ErrorKind;

fn main() {
    let greeting_file_result = File::open("hello3.txt");

    let greeting_file = match greeting_file_result {
        Ok(file) => file,
        Err(error) => match error.kind() {
            ErrorKind::NotFound => match File::create("hello33.txt") {
                Ok(fc) => fc,
                Err(e) => panic!("Problem creating the file: {:?}", e),
            },
            other_error => {
                panic!("Problem opening the file: {:?}", other_error);
            }
        },
    };
}

In [12]:
main()

()

## Ignore the error
- 코드를 프로토타이핑 중인데 오류 처리에 시간을 소비하고 싶지 않습니다.
- 오류가 발생하지 않을 것이라고 확신합니다.

### unwrap 메서드를 사용해서 실제값을 가져온다. 값이 없을 경우 패닉처리

- 파일이 존재한다는 것을 알고 있더라도 컴파일러는 이를 알 방법이 없습니다. 
- 따라서 unwrap을 사용하여 컴파일러에 우리를 신뢰하고 내부 값을 반환하도록 지시합니다. 

In [13]:
use std::fs;

fn main() {
  let content = fs::read_to_string("./Cargo.toml").unwrap();
  println!("{}", content)
}

### 성공적이 값이 없을 경우 패닉을 발생 

In [14]:
main()

thread '<unnamed>' panicked at 'called `Result::unwrap()` on an `Err` value: Os { code: 2, kind: NotFound, message: "No such file or directory" }', src/lib.rs:4:52
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::result::unwrap_failed
   3: <unknown>
   4: <unknown>
   5: <unknown>
   6: evcxr::runtime::Runtime::run_loop
   7: evcxr::runtime::runtime_hook
   8: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


## Terminate the program

- 일부 오류는 처리하거나 복구가 불가해서 프로그램을 종료하여 빠르게 종료해야 한다. 
- 반드시 존재하는 파일을 읽고 있다고 가정해 보면  이 프로그램에서 해당 파일이 없으면 제대로 작동하지 않고 바로 프로그램을 종료한다.
- unwrap과 동일하지만 추가 오류 메시지를 추가할 수 있는 expect를 사용할 수 있습니다.

In [15]:
use std::fs;

fn main() {
  let content = fs::read_to_string("./Cargo.toml").expect("Can't read Cargo.toml");
  println!("{}", content)
}

In [16]:
main()

thread '<unnamed>' panicked at 'Can't read Cargo.toml: Os { code: 2, kind: NotFound, message: "No such file or directory" }', src/lib.rs:4:52
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::result::unwrap_failed
   3: <unknown>
   4: <unknown>
   5: <unknown>
   6: evcxr::runtime::Runtime::run_loop
   7: evcxr::runtime::runtime_hook
   8: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


## Use a fallback value

- 경우에 따라 기본값으로 되돌아가서 오류를 처리할 수 있습니다.

- 예를 들어 서버를 작성할 때 서버가 수신 대기하는 포트를 환경 변수를 사용하여 구성할 수 있다고 가정해 보겠습니다. 
- 환경 변수가 설정되어 있지 않으면 해당 값에 액세스하면 오류가 발생합니다. 
- 하지만 기본값으로 되돌아가면 이 문제를 쉽게 처리할 수 있습니다.

In [17]:
use std::env;

fn main() {
  let port = env::var("PORT").unwrap_or("3000".to_string());
  println!("{}", port);
}

In [18]:
main()

3000


()

In [19]:
fn count(x: &str) -> usize { x.len() }


In [20]:
assert_eq!(Ok(2).unwrap_or_else(count), 2)

()

In [21]:
assert_eq!(Err("foo").unwrap_or_else(count), 3);

In [22]:
let good_year_from_input = "1909";
let bad_year_from_input = "190blarg";
let good_year = good_year_from_input.parse().unwrap_or_default();
let bad_year = bad_year_from_input.parse().unwrap_or_default();

assert_eq!(1909, good_year);
assert_eq!(0, bad_year);

## Bubble up the error

- 오류를 처리할 컨텍스트가 충분하지 않은 경우 오류를 호출자 함수에 버블업(전파)할 수 있습니다.

In [23]:
use std::fs::File;
use std::io::{self, Read};

fn read_username_from_file() -> Result<String, io::Error> {
    let username_file_result = File::open("hello4.txt");

    let mut username_file = match username_file_result {
        Ok(file) => file,
        Err(e) => return Err(e),
    };

    let mut username = String::new();

    match username_file.read_to_string(&mut username) {
        Ok(_) => Ok(username),
        Err(e) => Err(e),
    }
}

fn main() {
    let err_msg = read_username_from_file();
    println!(" {:?} ",err_msg);
}

In [24]:
main()

 Err(Os { code: 2, kind: NotFound, message: "No such file or directory" }) 


()